In [1]:
import spacy
import pandas as pd
import numpy as np
from spacy.matcher import PhraseMatcher

In [2]:
data = pd.read_json('data/restaurant.json')
spam = pd.read_csv('data/spam.csv')

In [3]:
data

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58
...,...,...,...,...,...,...,...,...,...
643007,0D__A1PkjvpG2pceebzLUA,H8E0U8CCaRA1mVJpd3mMDw,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Just go here and eat all of things! Penne a l...,2018-08-28 22:18:29
644110,dsuz6EEsT_JphWTdE6rOew,iqXgKGWHGsF1uwczPBLd5g,r5PLDU-4mSbde5XekTXSCA,5,0,0,0,Went there for lunch today and it was sooo goo...,2018-11-10 02:14:12
644370,7LROJHWjoq96I7Md8LIbQg,zGjZDf7ntd1SlIPCo5JhXg,r5PLDU-4mSbde5XekTXSCA,5,0,0,0,Defalco' s the best in Scottsdale. Great sandw...,2015-02-14 23:51:41
644531,yOLew_YdCmxpDDCdnm2Xgg,IQ8JZGg2po5YrcYqCBQQrw,r5PLDU-4mSbde5XekTXSCA,4,20,9,16,Defalco's is a great stop for all things Itali...,2018-05-24 20:31:21


In [4]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

In [50]:
data.loc[109]

review_id                                 lDJIaF4eYRF4F7g6Zb9euw
user_id                                   lb0QUR5bc4O-Am4hNq9ZGg
business_id                               r5PLDU-4mSbde5XekTXSCA
stars                                                          4
useful                                                         2
funny                                                          0
cool                                                           0
text           I used to work food service and my manager at ...
date                                         2013-01-27 17:54:54
Name: 109, dtype: object

In [5]:
index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

In [6]:
# load spacy model
nlp = spacy.blank('en')

In [7]:
# create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

In [9]:
# create PhraseMatcher object. The tokenizer is the first argument. use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

In [16]:
# create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

In [17]:
# add the item patterns to the matcher

matcher.add('MENU',          # just a name for the set of rules we're matching to
            None,            # special actions to take on matched words
            *menu_tokens_list)

In [18]:
# find matches in the review_doc
matches = matcher(review_doc)

In [22]:
matches

[(8291075388056826051, 2, 3),
 (8291075388056826051, 16, 17),
 (8291075388056826051, 58, 59)]

In [23]:
for match in matches:
    print(f'Token number {match[1]}: {review_doc[match[1]:match[2]]}')

Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


In [28]:
from collections import defaultdict

In [29]:
# item_ratings is a dictionary of lists. if a key doesn't exist in item_ratings,
# the key is added with an empty list as the value
item_ratings = defaultdict(list)

In [31]:
type(item_ratings)

collections.defaultdict

In [42]:
for idx, review in data.iterrows():
    
    doc = nlp(review.text)
    
    # using the matcher from the previous exercise
    matches = matcher(doc)
    
    # create a set of the items found in the review text
    found_items = set([doc[match[1]:match[2]] for match in matches])
    
    # update item_ratings with rating for each item in found_items
    # transform the item strings to lowercase to make it case insensitive
    for item in found_items:
        item_ratings[str(item).lower()].append(review.stars)

In [58]:
keys = [key for key in item_ratings]

In [73]:
pd.Series([np.mean(item_ratings[key]) for key in [key for key in item_ratings]]).idxmin()

21

In [76]:
keys[21]

'chicken cutlet'

In [77]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

In [79]:
item_counts = sorted(counts, key=counts.get, reverse=True)

In [81]:
for item in item_counts:
    print(f'{item:>25}{counts[item]:>5}')

                    pizza  358
                    pasta  255
                 meatball  163
              cheesesteak  146
                  calzone  110
                 eggplant   95
                  cannoli   89
             cheese steak   88
                  lasagna   83
                  purista   67
               prosciutto   63
             chicken parm   58
          italian sausage   57
             garlic bread   46
                  gnocchi   45
                spaghetti   41
                 calzones   38
                   pizzas   33
                   salami   32
            chicken pesto   30
             italian beef   29
                 tiramisu   27
                     ziti   26
            italian combo   22
         chicken parmesan   21
       chicken parmigiana   18
           mac and cheese   18
               portobello   18
                 pastrami   16
           chicken cutlet   11
         steak and cheese    9
               roast beef    7
       f

In [82]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

NameError: name 'mean_ratings' is not defined

In [6]:
nlp = spacy.load('en')

In [7]:
doc = nlp("Tea is healthy and calming, don't you think?")

In [14]:
for token in doc:
    print(token)

Tea
is
healthy
and
calming
,
do
n't
you
think
?


In [17]:
print(f'Token \t\tLemma \t\tStopword'.format('Token', 'Lemma', 'Stopword'))
print('-'*40)
for token in doc:
    print(f'{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}')

Token 		Lemma 		Stopword
----------------------------------------
Tea		tea		False
is		be		True
healthy		healthy		False
and		and		True
calming		calm		False
,		,		False
do		do		True
n't		not		True
you		-PRON-		True
think		think		False
?		?		False


In [18]:
from spacy.matcher import PhraseMatcher

In [19]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

In [21]:
terms = ['Galaxy Note', 'iPhone 11', 'iPhone XS', 'Google Pixel']
patterns = [nlp(text) for text in terms]
matcher.add('TerminologyList', None, *patterns)

In [22]:
text_doc = nlp("Glowing review overall, and some really interesting side-by-side "
               "photography tests pitting the iPhone 11 Pro against the "
               "Galaxy Note 10 Plus and last year’s iPhone XS and Google Pixel 3.")
matches = matcher(text_doc)
print(matches)

[(3766102292120407359, 17, 19), (3766102292120407359, 22, 24), (3766102292120407359, 30, 32), (3766102292120407359, 33, 35)]


In [28]:
match_id, start, end = matches[0]
print(nlp.vocab.strings[match_id], text_doc[start:end])

TerminologyList iPhone 11


In [3]:
spam

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
# create an empty model
nlp = spacy.blank('en')

In [5]:
# create the TextCategorizer with exclusive classes and 'bow' architecture
textcat = nlp.create_pipe('textcat', config={'exclusive_classes': True, 
                                             'architecture': 'bow'})

In [6]:
# add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [7]:
# add labels to text classifier
textcat.add_label('ham')
textcat.add_label('spam')

1

In [8]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}}
                for label in spam['label']]

In [9]:
train_data = list(zip(train_texts, train_labels))
train_data[:5]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}}),
 ('U dun say so early hor... U c already then say...',
  {'cats': {'ham': True, 'spam': False}}),
 ("Nah I don't think he goes to usf, he lives around here though",
  {'cats': {'ham': True, 'spam': False}})]

In [10]:
from spacy.util import minibatch

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

In [11]:
# create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)

In [12]:
# iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) but we need to
    # send separate lists for texts and labels to update().
    # This is a quick way to split a list of tuples into lists
    texts, labels = zip(*batch)
    nlp.update(texts, labels, sgd=optimizer)

In [13]:
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

In [14]:
losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # iterate through minibatches
    for batch in batches:
        # each batch is a list of (text, label) but we need to
        # send separate lists for texts and labels to update().
        # this is a quick way to split a list of tuples into lists
        texts, labels = zip(*batch)
        nlp.update(texts, labels, sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 0.4318974211906266}
{'textcat': 0.6474976200079823}
{'textcat': 0.7842154482108383}
{'textcat': 0.871668365141693}
{'textcat': 0.9280939272314639}
{'textcat': 0.9655779823256229}
{'textcat': 0.9939651726794686}
{'textcat': 1.0127976501288052}
{'textcat': 1.0275637671914504}
{'textcat': 1.0378531321124917}


In [15]:
texts = ["Are you ready fot the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA"]
docs = [nlp.tokenizer(text) for text in texts]

# use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)

print(scores)

[[9.9996209e-01 3.7902697e-05]
 [1.1491313e-02 9.8850864e-01]]


In [18]:
# from the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'spam']
